### Imports

In [ ]:
import torch

from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset, filter_agents_by_frames
from l5kit.dataset import EgoDataset
from l5kit.rasterization import build_rasterizer

from l5kit.cle.closed_loop_evaluator import ClosedLoopEvaluator, EvaluationPlan
from l5kit.cle.metrics import (CollisionFrontMetric, CollisionRearMetric, CollisionSideMetric,
                               DisplacementErrorL2Metric, DistanceToRefTrajectoryMetric)
from l5kit.cle.validators import RangeValidator, ValidationCountingAggregator
from l5kit.simulation.dataset import SimulationConfig
from l5kit.simulation.unroll import ClosedLoopSimulator
import os
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
from prettytable import PrettyTable
from l5kit.visualization.visualizer.zarr_utils import simulation_out_to_visualizer_scene
from l5kit.visualization.visualizer.visualizer import visualize
from bokeh.io import output_notebook, show
from l5kit.data import MapAPI

from ego_agent import EgoAgent

## Prepare data path and load cfg

By setting the `L5KIT_DATA_FOLDER` variable, we can point the script to the folder where the data lies.

Then, we load our config file with relative paths and other configurations (rasteriser, training params...).

In [ ]:
#@title Download L5 Sample Dataset and install L5Kit
import os
RunningInCOLAB = 'google.colab' in str(get_ipython())
if RunningInCOLAB:
    !wget https://raw.githubusercontent.com/lyft/l5kit/master/examples/setup_notebook_colab.sh -q
    !sh ./setup_notebook_colab.sh
    os.environ["L5KIT_DATA_FOLDER"] = open("./dataset_dir.txt", "r").read().strip()
else:
    print("Not running in Google Colab.")
    os.environ["L5KIT_DATA_FOLDER"] = "./tmp/l5kit_data"

In [ ]:
# set env variable for data
dm = LocalDataManager(None)
# get config
cfg = load_config_data("./config.yaml")

## Load the model

We load here two models:
- the `simulation_model.pt` will be used to control the agents around the SDV;
- the `ego_model.pt` will be used to control the SDV itself;

Clearly, nothing prevents us from replacing one model with the other (or with a completely different one, provided the input and output stay the same)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

simulation_model_path = "./tmp/simulation_model.pt"
simulation_model = torch.load(simulation_model_path).to(device)
simulation_model = simulation_model.eval()

ego_model_path = "./tmp/planning_model.pt"
ego_model = EgoAgent()
ego_model = ego_model.eval()

torch.set_grad_enabled(False)

## Load the evaluation data


In [ ]:
# ===== INIT DATASET
eval_cfg = cfg["val_data_loader"]
rasterizer = build_rasterizer(cfg, dm)
mapAPI = MapAPI.from_cfg(dm ,cfg)
eval_zarr = ChunkedDataset(dm.require(eval_cfg["key"])).open()
eval_dataset = EgoDataset(cfg, eval_zarr, rasterizer)
print(eval_dataset)

## Define some simulation properties
We define here some common simulation properties such as the length of the simulation and how many scene to simulate.

**NOTE: these properties have a significant impact on the execution time. We suggest you to increase them only if your setup includes a GPU**

In [ ]:
scenes_to_unroll = [0, 10, 20]
num_simulation_step_example1 = 20
num_simulation_step_example2 = 20

# Measuring ego metrics with simulated agents
The second set of metric we're interested in answer to the question _"are simulated agents helpful when evaluating SDV?"_

To this end, we can run a simulation with the following features:
- both agents and the SDV are simulated (so we can measure interactions between them);
- only agents close to the SDV are simulated (this is the only set we really care about in terms of interaction with the SDV);
- the set of simulated agents is variable and depends on the proximity of SDV (as the SDV moves in the scene, the agents proximity changes).

**Quantitatively**, we can measure how these agents affect the closed-loop metrics for the SDV by comparing the results with those from the log-replayed agents.

**Qualitatively**, we can inspect the resulting simulation overlayed with the original trajectories.

# Quantitative evaluation

We define here which metrics are of interest and how to evaluate each one of them. These metrics applies to the SDV only and are the same used in the [closed-loop planning notebook](../planning/closed_loop_test.ipynb).

If you want to know more about how each component (metrics, validators, aggregators) of the evaluation stack works, please refer to the source code

In [ ]:
metrics = [DisplacementErrorL2Metric(),
           DistanceToRefTrajectoryMetric(),
           CollisionFrontMetric(),
           CollisionRearMetric(),
           CollisionSideMetric()]

validators = [RangeValidator("displacement_error_l2_validator", DisplacementErrorL2Metric, max_value=30),
              RangeValidator("distance_ref_trajectory_validator", DistanceToRefTrajectoryMetric, max_value=4),
              RangeValidator("collision_front_validator", CollisionFrontMetric, max_value=0),
              RangeValidator("collision_rear_validator", CollisionRearMetric, max_value=0),
              RangeValidator("collision_side_validator", CollisionSideMetric, max_value=0),
              ]

intervention_validators = ["displacement_error_l2_validator",
                           "distance_ref_trajectory_validator",
                           "collision_front_validator",
                           "collision_rear_validator",
                           "collision_side_validator"]

cle_evaluator = ClosedLoopEvaluator(EvaluationPlan(metrics=metrics,
                                    validators=validators,
                                    composite_metrics=[],
                                    intervention_validators=intervention_validators))

### Evaluating with simulated agents
In this evaluation **both the models for planning and simulation are enabled**

In [10]:
sim_cfg = SimulationConfig(use_ego_gt=False, use_agents_gt=False, disable_new_agents=False,
                           distance_th_far=30, distance_th_close=15, num_simulation_steps=num_simulation_step_example2,
                           start_frame_index=0, show_info=True)

sim_loop = ClosedLoopSimulator(sim_cfg, eval_dataset, device, model_ego=ego_model, model_agents=simulation_model)

sim_outs = sim_loop.unroll(scenes_to_unroll)

cle_evaluator.evaluate(sim_outs)
validation_results = cle_evaluator.validation_results()
agg = ValidationCountingAggregator().aggregate(validation_results)
cle_evaluator.reset()

# Qualitative evaluation
Similarly to the previous example, also here we can show an interactive plot of the simulated scene. Because also the SDV is controlled now, its trajectory are shown in the plot

In [ ]:
output_notebook()
for sim_out in sim_outs: # for each scene
    vis_in = simulation_out_to_visualizer_scene(sim_out, mapAPI)
    show(visualize(sim_out.scene_id, vis_in))